In [1]:
! uv pip uninstall --system 'tensorflow'
! uv pip install --system --no-index --find-links='/kaggle/input/download-wheel-files/whls' 'pymupdf' 'vllm' 'triton' 'logits-processor-zoo' 'numpy<2'

Using Python 3.11.13 environment at: /usr
Uninstalled 1 package in 4.58s
 - tensorflow==2.18.0
Using Python 3.11.13 environment at: /usr
Resolved 161 packages in 312ms                                       
Prepared 55 packages in 12.26s                                           
Uninstalled 14 packages in 245ms
Installed 55 packages in 137ms                              
 + airportsdata==20250706
 + astor==0.8.1
 + blake3==1.0.5
 + compressed-tensors==0.9.3
 + depyf==0.18.0
 + diskcache==5.6.3
 + fastapi-cli==0.0.8
 + fastapi-cloud-cli==0.1.5
 + gguf==0.17.1
 + httptools==0.6.4
 - importlib-metadata==8.7.0
 + importlib-metadata==8.0.0
 + interegular==0.3.3
 + lark==1.2.2
 + llguidance==0.7.30
 - llvmlite==0.43.0
 + llvmlite==0.44.0
 + lm-format-enforcer==0.10.11
 + logits-processor-zoo==0.2.1
 + mistral-common==1.8.3
 + msgspec==0.19.0
 - numba==0.60.0
 + numba==0.61.2
 - nvidia-cublas-cu12==12.5.3.2
 + nvidia-cublas-cu12==12.4.5.8
 - nvidia-cuda-cupti-cu12==12.5.82
 + nvidia-cuda-cup

In [2]:
!uv pip install kagglehub more_itertools torch transformers

Using Python 3.11.13 environment at: /usr
Audited 4 packages in 172ms


In [3]:
import pandas as pd
import os

def is_submission():
    return os.getenv("KAGGLE_IS_COMPETITION_RERUN")

path = f"/kaggle/input/jigsaw-agile-community-rules/{'test.csv' if is_submission() else 'train.csv'}"

In [4]:
data = pd.read_csv(path)
data.head(1)

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",0


In [8]:
data['rule'][0]

'No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.'

### Steps for initial solution

- Step - 1 Add a column for expanding the rule by describing the given rule and make it understandable along with the prompt.
- Step - 2 Each row updated with prompt with positive and negative examples. 
- Step - 3 Then finally setup system prompt by describing the nature of the tool and get the otuput as voilation or nonvoilation along with other information.
            

In [5]:
import kagglehub
import more_itertools
import pandas as pd
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

In [7]:
GEMMA_PATH = kagglehub.model_download("google/gemma-3/transformers/gemma-3-1b-it")
processor = AutoTokenizer.from_pretrained(GEMMA_PATH)
device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(GEMMA_PATH).to(device)

In [30]:
prompts = ["Hello my name is sachin.", "How are you?"]

In [31]:
### prompt template for exmapnding the rule for better contex addition.

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2714: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [43]:
### SYS_MSG_EXPLORATION

SYS_PROMPT = """
<SYS>
You are an expert with understanding of community guidlines and related keywords.
For a given community rule is provided, you go through its keywords and exapand each keyword in details. 

Output should only contain the expanded rules no explanatory text.
"""

In [44]:
rule = data['rule'][0]

In [45]:
user_prompt = f"""
<USER>
Expand below rule

{rule}

"""

In [47]:
prompt_text = (
    SYS_PROMPT + "\n\n" +
    user_prompt
)
inputs = processor(prompt_text, return_tensors="pt").to(device)

In [48]:
print(prompt_text)


<SYS>
You are an expert with understanding of community guidlines and related keywords.
For a given community rule is provided, you go through its keywords and exapand each keyword in details. 

Output should only contain the expanded rules no explanatory text.



<USER>
Expand below rule

No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.




In [49]:
device

'cuda'

In [50]:
generated = model.generate(**inputs, max_length=200)

# Convert token IDs back to text
decoded_text = processor.batch_decode(generated, skip_special_tokens=True)
print(decoded_text)

["\n<SYS>\nYou are an expert with understanding of community guidlines and related keywords.\nFor a given community rule is provided, you go through its keywords and exapand each keyword in details. \n\nOutput should only contain the expanded rules no explanatory text.\n\n\n\n<USER>\nExpand below rule\n\nNo Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.\n\n<SYS>\n<SYS>\n**No Advertising:** Spam, referral links, unsolicited advertising, and promotional content are not allowed.\n\n*   **Spam:**  Spam is defined as unsolicited, repetitive, and often deceptive messages. It's a violation of community guidelines because it disrupts the flow of conversation and can be harmful to users.\n*   **Referral links:** Referral links are links that direct users to another website or service. They are often used to promote products or services, and can be considered intrusive if they are not disclosed.\n*   "]


In [52]:
output = decoded_text[0]

In [63]:
import re

replace_str = prompt_text + "|<SYS>"

updated_rule = re.sub(replace_str, "", output)

In [64]:
data.columns

Index(['row_id', 'body', 'rule', 'subreddit', 'positive_example_1',
       'positive_example_2', 'negative_example_1', 'negative_example_2',
       'rule_violation'],
      dtype='object')

In [65]:
rule = updated_rule
target_comment = data['rule'][0]
positive_comment1 = data['positive_example_1'][0]
positive_comment2 = data['positive_example_2'][0]
negative_comment1 = data['negative_example_1'][0]
negative_comment2 = data['negative_example_2'][0]

In [70]:
SYSTEM_PROMPT_CLASSIFICATION = """

<SYS>
you are a community rule voilation inspector. 

if you are give the rule with context and target comment along with negative and positive comments you can decide if the target comment is in voilation of the specific rule.

you will use positive comment to understand how those comments voilates the rule and negative comments to understand why negative comments are not breaking the rules,

you are required to answer only one of the 2 values, 0 or 1. 0 denotes "not voilation" and 1 denotes "voilation".

output should be given as: 

{
    voilation: 1 or 0,
    confidence_score: <between 0 and 1, higher the better.>,
    justification: <reason why specific voilation value was choosen.>

}

"""


USER_PROMPT_CLASSIFICATION = f"""
<USER>
Classify the below comment based on the rule given, 

comment = {target_comment}

below are the details related to the comment: 

Rule : {rule}
positive_comment1 = {positive_comment1}
positive_comment2 = {positive_comment2}
negative_comment1 = {negative_comment1}
negative_comment2 = {negative_comment2}

"""


prompt_text_classifier = (
    SYSTEM_PROMPT_CLASSIFICATION + "\n\n" +
    USER_PROMPT_CLASSIFICATION
)

In [74]:
# print(prompt_text_classifier)

In [75]:
classifier_inputs = processor(prompt_text, return_tensors="pt").to(device)

In [76]:
generated_classification = model.generate(**classifier_inputs, max_length=500)

# Convert token IDs back to text
decoded_classifier = processor.batch_decode(generated_classification, skip_special_tokens=True)
print(decoded_classifier[0])


<SYS>
You are an expert with understanding of community guidlines and related keywords.
For a given community rule is provided, you go through its keywords and exapand each keyword in details. 

Output should only contain the expanded rules no explanatory text.



<USER>
Expand below rule

No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.

<SYS>
<SYS>
**No Advertising:** Spam, referral links, unsolicited advertising, and promotional content are not allowed.

*   **Spam:**  Spam is defined as unsolicited, repetitive, and often deceptive messages. It's a violation of community guidelines because it disrupts the flow of conversation and can be harmful to users.
*   **Referral links:** Referral links are links that direct users to another website or service. They are often used to promote products or services, and can be considered intrusive if they are not disclosed.
*   **Unsolicited advertising:** Unsolicited advertising is advertis